# Notebook to demonstrate Time Series data processing with Pandas



It includes steps for :
    1. Resampling a dataframe so that every date has some value (With Forward fill)
    2. Extracting a subset of rows based on date range 
    3. Ploting a dataframe
    4. Merging two dataframes 

In [ ]:
import pandas as pd

In [ ]:
def get_data_dir():
    return '../data/'


def macro_raw_dir():
    return get_data_dir()+'macro/raw/'


def stock_dir():
    return get_data_dir()+'stock/'


def gold_price_file():
    return macro_raw_dir()+'gold_rates_30_year.xlsx'


def stock_price_file(symbol):
    return stock_dir()+symbol+'_Excel.xlsx'


def merged_file_path():
    return get_data_dir()+'Merged_Data_Excel_Test.xlsx'


def us_treasury_yield_file_processed():
    return get_data_dir()+'macro/'+'us_daily_treasury_yield_1990.xlsx'


def fill_and_subset_timeseries(df):
    df = df.sort_index()
    df = df.asfreq('D',method='ffill')
    df = df.loc['1995-01-01':'2019-05-31']
    df = df.sort_index()
    return df

## Read Gold price data

In [ ]:
gold_file = gold_price_file()
#gold_file

In [ ]:
gold_df = pd.read_excel(gold_file,index_col=0,parse_dates=True )

In [ ]:
#gold_df

## Read Stock price data

In [ ]:
stock='KO'
stock_file=stock_price_file(stock)

stock_df = pd.read_excel(stock_file,index_col=0,parse_dates=True )

## Initialize Visualization libraries

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf

In [ ]:
init_notebook_mode(connected=True)
cf.go_offline()

## Plot Prices (+ Copy of stock price without volume)

In [ ]:
gold_df.iplot()

In [ ]:
stock_df.iplot()

### Drop volume and 'Adjusted Price' columns from stock prices and plot

In [ ]:
stock_price_only_df = stock_df.drop(['Volume','Adj Close'], axis=1)

In [ ]:
stock_price_only_df.iplot()

## Pre-process data 

### Counte rows in dataframes

In [ ]:
stock_price_only_df.count()
gold_df.count()

### Ensure every date has a value, use Forward-Fill to generae values

In [ ]:
stock_price_only_df = stock_price_only_df.asfreq('D',method='ffill')

### Validate with count

In [ ]:
stock_price_only_df.count()

### Create a subset (Based on date Range)

In [ ]:
stock_price_only_df=stock_price_only_df.loc['1995-01-01':'2018-12-31']

In [ ]:
stock_price_only_df.count()

In [ ]:
stock_price_only_df.iplot()

### Pre-process with defined funtion

In [ ]:
stock_prices=fill_and_subset_timeseries(stock_df)

In [ ]:
stock_prices.count()

In [ ]:
stock_df.count()

In [ ]:
gold_prices=fill_and_subset_timeseries(gold_df)

In [ ]:
gold_prices.count()

### Merge (Join) two datasets

In [ ]:
merged_df=stock_prices.join(gold_prices)

In [ ]:
merged_df.count()

In [ ]:
merged_df.iplot()

In [ ]:
### Drop Adjusted Close Price

In [ ]:
merged_df=merged_df.drop(['Adj Close'], axis=1)

## Load US Treasury Rates

In [ ]:
 treasury_df = pd.read_excel(us_treasury_yield_file_processed(),index_col=0,parse_dates=True )

In [ ]:
treasury_df=fill_and_subset_timeseries(treasury_df)
treasury_df.iplot()

In [ ]:
merged_df=merged_df.join(treasury_df)

In [ ]:
merged_df['days_since_epoch']= (merged_df.index - pd.Timestamp('1970-01-01')).days

In [ ]:
merged_df.head(10)

In [ ]:
merged_df.to_excel(merged_file_path())